<a href="https://colab.research.google.com/github/tb268/Atcoder/blob/main/vit%E5%8D%92%E7%A0%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os
# フォルダを作成する
os.makedirs('./data')


In [2]:
#フォルダ削除用
import shutil
shutil.rmtree('/content/data')

# フォルダを作成する
os.makedirs('./data')

In [3]:
!pip install reformer-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached 

In [4]:
!nvidia-smi

Wed May 15 07:30:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [23]:
# reformer版 3D Pose Estimation
import torch
import torch.nn as nn
from reformer_pytorch import Reformer, Autopadder

class PoseReformer3D(nn.Module):
    def __init__(self, patch_size=32, num_keypoints=17, dim=512, depth=6, heads=8, bucket_size=16, n_hashes=4):
        super(PoseReformer3D, self).__init__()
        self.patch_size = patch_size
        self.num_keypoints = num_keypoints
        self.dim = dim
        # パッチ埋め込み (Patch Embedding)
        self.patch_to_embedding = nn.Linear(3 * patch_size * patch_size, dim)
        # クラストークン (Class Token)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        # Reformer モデルの定義
        self.reformer = Reformer(
            dim=dim,
            depth=depth,
            heads=heads,
            bucket_size=bucket_size,
            n_hashes=n_hashes
        )
        self.reformer = Autopadder(self.reformer)
        self.to_cls_token = nn.Identity()
        # 最終的なMLPヘッド
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_keypoints * 3)  # 3次元座標 (x, y, z) なので num_keypoints * 3
        )

    def forward(self, img):
        B, C, H, W = img.shape
        p = self.patch_size
        # 画像ピクセルをパッチサイズに分割
        x = img.unfold(2, p, p).unfold(3, p, p)  # (B, C, H/p, W/p, p, p)
        x = x.permute(0, 2, 3, 1, 4, 5)  # (B, H/p, W/p, C, p, p)
        x = x.reshape(x.shape[0], -1, C * p * p)  # (B, num_patches, patch_dim)
        num_patches = x.shape[1]
        # パッチ埋め込み (Patch Embedding)
        x = self.patch_to_embedding(x)  # (B, num_patches, dim)

        # クラストークン (Class Token)
        cls_tokens = self.cls_token.expand(B, -1, -1)  # (B, 1, dim)
        x = torch.cat((cls_tokens, x), dim=1)  # (B, num_patches + 1, dim)

        # 位置埋め込み (Position Embedding)
        pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, self.dim))
        x += pos_embedding[:, :(x.size(1))]

        # Reformer での処理
        x = self.reformer(x)  # (B, num_patches + 1, dim)
        x = self.to_cls_token(x[:, 0])  # クラストークンの取得
        # 最終的なMLPヘッドでキーポイントを予測
        keypoints = self.mlp_head(x)  # (B, num_keypoints * 3)
        keypoints = keypoints.view(x.size(0), -1, 3)  # (B, num_keypoints, 3)
        return keypoints

def generate_dummy_data(num_samples, num_keypoints, img_height, img_width):
    images = torch.randn(num_samples, 3, img_height, img_width)  # ダミー画像データ
    keypoints = torch.randn(num_samples, num_keypoints, 3) * 1000  # ダミー3Dキーポイント（mm単位）
    return images, keypoints

def compute_mpjpe(predicted, true):
    return torch.mean(torch.norm(predicted - true, dim=-1))  # 各キーポイント間の距離を計算し、平均を取る

# ダミーデータ生成
num_samples = 100
num_keypoints = 17
img_height, img_width = 320, 240  # 任意の画像サイズ
images, true_keypoints = generate_dummy_data(num_samples, num_keypoints, img_height, img_width)

# モデルのインスタンス化
model = PoseReformer3D()
model.eval()  # 評価モードに切り替え

# 推論の実行
with torch.no_grad():
    predicted_keypoints = model(images)

# MPJPEの計算
mpjpe = compute_mpjpe(predicted_keypoints, true_keypoints)
print(f'Average MPJPE: {mpjpe.item():.2f} mm')


Average MPJPE: 1625.53 mm


In [22]:
# transformer版 3D Pose Estimation
import torch
import torch.nn as nn
import numpy as np
from torch.nn import Transformer
class PoseTransformer3D(nn.Module):
    def __init__(self, image_size=256, patch_size=32, num_keypoints=17, dim=512, depth=6, heads=8):
        super(PoseTransformer3D, self).__init__()
        assert image_size % patch_size == 0, 'Image size must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = 3 * patch_size * patch_size

        self.patch_size = patch_size
        # 位置埋め込み (Position Embedding)
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))  # +1 for class token
        # パッチ埋め込み (Patch Embedding)
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        # クラストークン (Class Token)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        # Transformer モデルの定義
        self.transformer = Transformer(
            d_model=dim,
            nhead=heads,
            num_encoder_layers=depth,
            num_decoder_layers=depth,
            dim_feedforward=dim * 4,
            dropout=0.1
        )
        self.to_cls_token = nn.Identity()
        # 最終的なMLPヘッド
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_keypoints * 3)  # 3次元座標 (x, y, z) なので num_keypoints * 3
        )

    def forward(self, img):
        p = self.patch_size
        # 画像ピクセルを32x32のパッチに分割
        x = img.unfold(2, p, p).unfold(3, p, p)  # (B, C, H/p, W/p, p, p)
        x = x.permute(0, 2, 3, 1, 4, 5)  # (B, H/p, W/p, C, p, p)
        x = x.reshape(x.shape[0], -1, 3 * p * p)  # (B, num_patches, patch_dim)
        # パッチ埋め込み (Patch Embedding)
        x = self.patch_to_embedding(x)  # (B, num_patches, dim)

        # クラストークン (Class Token)
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)  # (B, 1, dim)
        x = torch.cat((cls_tokens, x), dim=1)  # (B, num_patches + 1, dim)

        # 位置埋め込み (Position Embedding)
        x += self.pos_embedding[:, :(x.size(1))]

        # Transformer での処理
        x = self.transformer(x, x)  # (B, num_patches + 1, dim)
        x = self.to_cls_token(x[:, 0])  # クラストークンの取得
        # 最終的なMLPヘッドでキーポイントを予測
        keypoints = self.mlp_head(x)  # (B, num_keypoints * 3)
        keypoints = keypoints.view(x.size(0), -1, 3)  # (B, num_keypoints, 3)
        return keypoints
def generate_dummy_data(num_samples, num_keypoints):
    images = torch.randn(num_samples, 3, 256, 256)  # ダミー画像データ
    keypoints = torch.randn(num_samples, num_keypoints, 3) * 1000  # ダミー3Dキーポイント（mm単位）
    return images, keypoints

num_samples = 100
num_keypoints = 17
images, true_keypoints = generate_dummy_data(num_samples, num_keypoints)
def compute_mpjpe(predicted, true):
    return torch.mean(torch.norm(predicted - true, dim=-1))  # 各キーポイント間の距離を計算し、平均を取る
# モデルのインスタンス化
model = PoseTransformer3D()
model.eval()  # 評価モードに切り替え

# 推論の実行
with torch.no_grad():
    predicted_keypoints = model(images)

# MPJPEの計算
mpjpe = compute_mpjpe(predicted_keypoints, true_keypoints)
print(f'Average MPJPE: {mpjpe.item():.2f} mm')


Average MPJPE: 1576.79 mm
